In [50]:
import sys
import fitz
import requests
import json
import numpy as np

# Newer tests with rect-based tagging

Should in theory be more robust and less error prone.

In [51]:
ppd_blue = (115./255., 203./255., 235./255.)

In [52]:
def add_ppd_box(page,rect) : 
    
    page.addRedactAnnot(rect)
    
    # Need to also add "PPD" in the rectangle
    rc    = page.drawRect(rect,color=ppd_blue,fill=ppd_blue)
    shape = page.newShape()

    # rc.setColor(ppd_blue)
    rc    = shape.insertTextbox(rect, "PPD", color = (0,0,0))
    shape.commit()
    
    page.apply_redactions()

In [53]:
def mark_words_with_rect(page,tags) : 

    tokens = tags['tokens']
    rects  = tags['rects']

    for i in range(len(tokens)) :
        for j in range(len(tokens[i])) : 
            
            value = tokens[i][j]['token']
            label = tokens[i][j]['label']

            for punct in ".,!@?()[]-" :
                value = value.replace(" "+punct,punct)
            
            if label == 'O' :
                continue
                
            # print("Tagging value:",value)
            
            rect  = [float(ri) for ri in rects[i][j]]
            r = fitz.Rect(rect)       # make rect from word bbox
            
            # We've crossed over - try to find the annotation instead
            if rect[0]>rect[2] and rect[3]>rect[1] :
                search_value = value
                print("Looking for multi-line phrase: %s"%(search_value))
                ql = page.searchFor(search_value)
                print(rect)
                print(ql)              
                for iq in range(len(ql)) :                   
                    if abs(float(ql[iq][0])-rect[0])<0.01 :
                        # Hard defaulting to only assuming one line of overlap
                        rect = [ql[iq], ql[iq+1]]
                        print("Got rect:",rect)
                        break

                for ir in rect :
                    add_ppd_box(page,ir)
                        
                continue              
            
            add_ppd_box(page,rect)

In [54]:
fname = "doc2.pdf"
doc   = fitz.open(fname)

for page in doc :
    
    # NOTE: getTextWords also returns rect - we can use this probably
    sentence = " ".join([word[4] for word in page.getTextWords()])
    # print(sentence)
    
    npp    = np.array(page.getTextWords())
    words  = npp[:,4].tolist()
    rects  = npp[:,0:4].tolist()
    
    url   = 'http://127.0.0.1:5050/tag_sentence_rects'
    myobj = {'tokens': words, 'rects' : rects}
    # print(myobj)
    x      = requests.post(url=url,json=myobj)
    tags   = json.loads(x.text)['result']
    
    # print(tags)
    
    mark_words_with_rect(page, tags)
    
doc.save("redacted-" + doc.name)

Looking for multi-line phrase: seibei nakagawa
[463.3050842285156, 86.52978515625, 127.03556060791016, 110.52978515625]
[Rect(463.30517578125, 86.52978515625, 496.9412841796875, 97.52978515625), Rect(72.0, 99.52978515625, 123.9788818359375, 110.52978515625)]
Got rect: [Rect(463.30517578125, 86.52978515625, 496.9412841796875, 97.52978515625), Rect(72.0, 99.52978515625, 123.9788818359375, 110.52978515625)]
Looking for multi-line phrase: sapporo beer sapporo
[475.57342529296875, 359.52978515625, 138.66387939453125, 383.52978515625]
[Rect(475.5734558105469, 359.52978515625, 520.2205810546875, 370.52978515625), Rect(72.0, 372.52978515625, 138.66387939453125, 383.52978515625)]
Got rect: [Rect(475.5734558105469, 359.52978515625, 520.2205810546875, 370.52978515625), Rect(72.0, 372.52978515625, 138.66387939453125, 383.52978515625)]
